# Read a MySQL DataBase and generate specific output

In [70]:
import getpass
import pymysql
import unidecode
import pandas as pd
import tempfile
import subprocess
import sys

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',500)

def generate_pdf(row):
    
    if row.opening is  None:
        row.opening=''
    if row.closing is  None:
        row.closing=''

    latexfile='\n'
    latexfile=latexfile+'%'+row.description+'\n'
    latexfile=latexfile+r'\documentclass{letter}'+'\n'
    latexfile=latexfile+r'\usepackage[utf8]{inputenc}'+'\n'
    latexfile=latexfile+row.preamble+'\n'
    latexfile=latexfile+row.address+'\n'
    latexfile=latexfile+r'\signature{'+row.sign+'}\n'
    latexfile=latexfile+r'\begin{document}'+'\n'
    latexfile=latexfile+r'\begin{letter}{'+row.para+'}\n'
    latexfile=latexfile+r'\opening{'+row.opening+'}\n'
    latexfile=latexfile+row.body+'\n'
    latexfile=latexfile+r'\closing{'+row.closing+'}\n'
    latexfile=latexfile+r'\end{letter}'+'\n'
    latexfile=latexfile+r'\end{document}'+'\n'

    f=tempfile.NamedTemporaryFile('w',suffix='.tex',delete=False)
    f.write(latexfile)
    f.close()
    lo=subprocess.Popen('pdflatex {:s}'.format(f.name).split(),cwd='/tmp',
                        stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
    pdffile='{:s}'.format(f.name).split('.tex')[0]+'.pdf'
    if lo[0].decode('utf8').find(pdffile.split('/')[-1])>-1:
        pdf='carta_{:d}.pdf'.format(row['id'])
        dwld_dir='downloads'
        kk=subprocess.Popen('mv {:s} /home/restrepo/{:s}/{:s}'.format(pdffile,dwld_dir,pdf).split()).communicate()
        print('output in http://fisica.udea.edu.co/{:s}/{:s}'.format(dwld_dir,pdf))
        for suffix in  ['.aux','.log','.out','.tex']:
            subprocess.Popen( ['rm', '{:s}'.format(f.name).split('.tex')[0]+suffix] ).communicate()
    else:
        sys.exit('LaTeX ERROR:')    
    return latexfile

def db_to_pdf(choose_db=0,id=None,verbose=False):
    '''
    Obtain letter from the last DB entry
    Type of letter: 
      * 0 Personal
      * 1 Group
    '''
    password=getpass.getpass()
    tipo_carta=['Diego_Restrepo', 'cartas_gfif']
    conn = pymysql.connect(host='localhost', port=3306, user='root', passwd=password, db=tipo_carta[choose_db])

    df=pd.read_sql('select * from cartas;', con=conn)

    sg=pd.read_sql('select * from cartas_sign;', con=conn)
    
    
    conn.close()
    
    #print(df.shape[0])
    #return df,sg
    if df.shape[0]:
        if not id:
            id=df.id.values[-1]
        c=df[df.id==id].reset_index(drop=True).loc[0]
        c['sign']=sg[sg.signature==c.signature].reset_index(drop=True).loc[0,'sign']
        if verbose:
            print(c)
    else:
        return df

    
    
    return generate_pdf(c)

def backup_database(choose_db=0):
    '''
    Obtain letter from the last DB entry
    Type of letter: 
      * 0 Personal
      * 1 Group
    '''
    password=getpass.getpass()
    tipo_carta=['Diego_Restrepo', 'cartas_gfif']
    
    print('Trying to create backup file db_{:s}_backup.sql'.format( tipo_carta[choose_db]  )   )
    ok=subprocess.Popen('mysqldump -u root -p{:s} {:s} > db_{:s}_backup.sql'.format(
                     password,tipo_carta[choose_db],tipo_carta[choose_db]),shell=True,
                     stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
    
    print()
    
    return ok

In [71]:
choose_db=0
latexfile=db_to_pdf(choose_db,id=272)

········
output in http://fisica.udea.edu.co/downloads/carta_272.pdf


In [72]:
#print(latexfile)

### Backup database

In [73]:
backup_database(choose_db)

········
Trying to create backup file db_Diego_Restrepo_backup.sql



(b'', b'')

## Manual

In [13]:
password=getpass.getpass()

········


In [14]:
choose_db=0
tipo_carta=['Diego_Restrepo', 'cartas_gfif']

conn = pymysql.connect(host='localhost', port=3306, user='root', passwd=password, db=tipo_carta[choose_db])

df=pd.read_sql('select * from cartas;', con=conn)

sg=pd.read_sql('select * from cartas_sign;', con=conn)

In [15]:
conn.close()

In [29]:
df=df[df.body.str.lower().str.contains('julian ')]

Last entry

In [30]:
df.id.values[-1]

272

In [31]:
c=df[df.id==df.id.values[-1]].reset_index(drop=True).loc[0]
c['sign']=sg[sg.signature==c.signature].reset_index(drop=True).loc[0,'sign']

In [ ]:
c

In [39]:
if  c.opening:
    print(1)

In [42]:
row.description

'CERN Studentships in Summer 2017'

In [43]:
row=c

In [ ]:
if row.opening is  None:
    row.opening=''
if row.closing is  None:
    row.closing=''

latexfile='\n'
latexfile=latexfile+'%'+row.description+'\n'
latexfile=latexfile+r'\documentclass{letter}'+'\n'
latexfile=latexfile+r'\usepackage[utf8]{inputenc}'+'\n'
latexfile=latexfile+row.preamble+'\n'
latexfile=latexfile+row.address+'\n'
latexfile=latexfile+r'\signature{'+row.sign+'}\n'
latexfile=latexfile+r'\begin{document}'+'\n'
latexfile=latexfile+r'\begin{letter}{'+row.para+'}\n'
if row.opening:
    latexfile=latexfile+r'\opening{'+row.opening+'}\n'
latexfile=latexfile+row.body+'\n'
latexfile=latexfile+r'\closing{'+row.closing+'}\n'
latexfile=latexfile+r'\end{letter}'+'\n'
latexfile=latexfile+r'\end{document}'+'\n'
    
print(latexfile)

In [20]:
c.opening

'Respetados señores:'

In [21]:
c.sign

"\\includegraphics[scale=0.4]{firma}\\\\\r\n\\noindent\r\nDiego A. Restrepo Quintero\\\\\r\nCoordinador\\\\\r\nGrupo de Fenomenología de Interacciones Fundamentales\\\\\r\nInstituto de F\\'\\i sica\\\\\r\nUniversidad de Antioquia"

In [18]:
ok

(b'', b'')

In [10]:
'mysqldump -u root -p{:s} {:s} > db_{:s}_backup.sql'.format(password,tipo_carta[NTC],tipo_carta[NTC]).split()

['mysqldump',
 '-u',
 'root',
 '-ppoioiulkj',
 'cartas_gfif',
 '>',
 'db_cartas_gfif_backup.sql']

## Search in body

In [50]:
id=99
if id >0:
    dfs=df[df.id==id].reset_index(drop=True)
else:    
    dfs=df[df.body.str.lower().map(unidecode.unidecode).str.contains('impacto social')].reset_index(drop=True)

In [ ]:
dfs

In [52]:
c=dfs.loc[dfs.index.values[-1]]

In [ ]:
c

In [ ]:
c.keys().values

In [ ]:
for k in c.keys():
    print(k)
    if c[k]==None:
        c[k]=''

In [52]:
c.closing

In [34]:
c.closing

'Best regards'

In [35]:
latexfile=r'''
%'''+c.description+r'''
\documentclass{letter}
\usepackage[utf8]{inputenc}
'''+c.preamble+r'''
'''+c.address+r'''
\signature{'''+sg[sg.signature==c.signature].sign.values[0]+r'''}
\begin{document}
\begin{letter}{'''+c.para+r'''}

\opening{'''+c.opening+r'''} 

'''+c.body+r'''

\closing{'''+c.closing+r'''}
\end{letter}
\end{document}
'''

TypeError: Can't convert 'NoneType' object to str implicitly

In [ ]:
print(latexfile)

In [58]:
f=tempfile.NamedTemporaryFile('w',suffix='.tex',delete=False)

In [59]:
f.write(latexfile)

2017

In [60]:
f.name

'/tmp/tmppt2pwnu1.tex'

In [61]:
f.close()

In [62]:
lo=subprocess.Popen('pdflatex {:s}'.format(f.name).split(),cwd='/tmp',stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

In [ ]:
pdffile='{:s}'.format(f.name).split('.tex')[0]+'.pdf'
if lo[0].decode('utf8').find(pdffile.split('/')[-1])>-1:
    pdf='carta_{:d}.pdf'.format(c['id'])
    dwld_dir='downloads'
    kk=subprocess.Popen('mv {:s} /var/www/{:s}/{:s}'.format(pdffile,dwld_dir,pdf).split()).communicate()
    print('output in http://gfif.udea.edu.co/{:s}/{:s}'.format(dwld_dir,pdf))
    for suffix in  ['.aux','.log','.out','.tex']:
        subprocess.Popen( ['rm', '{:s}'.format(f.name).split('.tex')[0]+suffix] ).communicate()
else:
    sys.exit('LaTeX ERROR:')    